In [4]:
import random
import numpy as np
import cv2
import copy

lab_boundaries = [([190, 120, 138], [240, 138, 177])]
[(lower,upper)] = lab_boundaries
lower = np.array(lower, dtype="uint8")
upper = np.array(upper, dtype="uint8")

fourcc = cv2.VideoWriter_fourcc(*'XVID')

cap = cv2.VideoCapture('day2_lane1/0.jpg')
frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('test.avi', fourcc, 30.0,(frame_width,frame_height))
out2 = cv2.VideoWriter('test2.avi', fourcc, 30.0,(frame_width,frame_height))

bounds = [300, 600]

In [5]:
def find_first(item, vec):
    """return the index of the first occurence of item in vec"""
    for i in range(len(vec)):
        if item == vec[i]:
            return i
    return -1

In [6]:
def main():
    """main fuction
    """
    i = 0
    while i < 1000:
        fname = 'day2_lane1/' + str(i) + '.jpg'
        
        original = cv2.imread(fname, cv2.IMREAD_COLOR)
        filtered = copy.deepcopy(original)
        filtered = cv2.bilateralFilter(filtered,3,75,75)
        
        lab_image = cv2.cvtColor(filtered, cv2.COLOR_BGR2LAB)

        mask = cv2.inRange(lab_image, lower, upper)
        output = cv2.bitwise_and(lab_image, lab_image, mask=mask)
        
        canny = copy.deepcopy(output)     
        canny = cv2.Canny(canny,100,200)     
              
        window = canny[bounds[1]:bounds[0]:-1].transpose()
        xy = []
        for k in range(len(window)):
            col = window[k]
            j = find_first(255, col)
            if j != -1:
                xy.extend((k, j))
        # Reshape into [[x1, y1],...]
        data = np.array(xy).reshape((-1, 2))
        # Translate points back to original positions.
        data[:, 1] = bounds[1] - data[:, 1]
        pts = data.reshape((-1, 1, 2)) 
        
        output2 = cv2.polylines(original, [pts], isClosed=False, color=(255,0,0))

        out.write(output)
        out2.write(output2)
        i = i+1
#         c=cv2.waitKey(0)
        
#     cv2.destroyAllWindows()
    out.release()
    out2.release()

In [7]:
if __name__ == '__main__':
    main()